In [ ]:
import netCDF4 as nc
from spectral.io import envi
import matplotlib.pyplot as plt
import numpy as np
from isofit.core.common import resample_spectrum
import os
import isofit
#import tkinter
#import matplotlib
#matplotlib.use('TkAgg')
from isofit.core.sunposition import sunpos
from datetime import datetime

In [ ]:
obs_header = envi.open('../EMIT-scenes/west_africa_scene/emit20230321t150833_o08010_s000_l1b_obs_b0106_v01.hdr')
location_header = envi.open('../EMIT-scenes/west_africa_scene/emit20230321t150833_o08010_s000_l1b_loc_b0106_v01.hdr')
rad_header = envi.open('../EMIT-scenes/west_africa_scene/EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance.hdr')
mask_header = envi.open('../EMIT-scenes/west_africa_scene/EMIT_L2A_MASK_001_20230321T150833_2308010_003_mask.hdr')

obs_data = obs_header.open_memmap(interleave = 'bip')
loc_data = location_header.open_memmap(interleave = 'bip')
rad_data = rad_header.open_memmap(interleave = 'bip')
mask_data = mask_header.open_memmap(interleave = 'bip')

lat = loc_data[:,:,0]
lon = loc_data[:,:,1]
elev = loc_data[:,:,2]
zenith = obs_data[:,:,4]
zen = np.average(zenith)
zen = np.deg2rad(zen)
es_distance = obs_data[:,:,10]
water_vapor = mask_data[:,:,6]

wl = rad_header.metadata['wavelength']
fwhm = rad_header.metadata['fwhm']
for i in range(len(wl)):
    wl[i] = float(wl[i])
    fwhm[i] = float(fwhm[i])
wl = np.array(wl)
fwhm = np.array(fwhm)

"""
fwhm = rad_header.metadata['fwhm']
for i in range(len(fwhm)):
    fwhm[i] = float(fwhm[i])
fwhm = np.array(fwhm)

"""
irr_path = '../irr.npy'
data = np.load(irr_path)
irr_wl = data[:,0]
irr = data[:,1]

irr_resamp = resample_spectrum(irr, irr_wl, wl, fwhm)
irr_resamp = np.array(irr_resamp, dtype=np.float32)
irr = irr_resamp
irr = irr * ((np.average(es_distance))**2)

print('shape of latitude: ', lat.shape)
print('shape of longitude: ', lon.shape)
print('shape of elevation: ', elev.shape)
print('shape of zenith: ', zen.shape)
print('shape of earth-sun distance', es_distance.shape)
print('shape of water vapor: ', water_vapor.shape)
print('shape of wavelength: ', wl.shape)
print('shape of fwhm: ', fwhm.shape)
print('shape of irradiance: ', irr.shape)
print('shape of radiance: ', rad_data.shape)

"""
flight_name = rad_header.metadata['flight_line'][0]

year = flight_name[4:8]
month = flight_name[8:10]
date = flight_name[10:12]
hour = flight_name[13:15]
minute = flight_name[15:17]
second = flight_name[17:19]

dt_string = ''
dt_string = dt_string + date + '/' + month + '/' + year + ' ' + hour + ':' + minute + ':' + second

dt = datetime.strptime(dt_string, "%d/%m/%Y %H:%M:%S")
print(dt)

es_path = os.path.join(os.path.dirname(isofit.__file__), "..", "data", "earth_sun_distance.txt")
ref_es = np.loadtxt(es_path)

"""

In [ ]:
import time

start = time.time()
rho = (np.pi / np.cos(zen)) * (rad_data / irr[np.newaxis, np.newaxis, :])
end = time.time()
elapsed = end - start
print(elapsed, ' seconds')
print(rho.shape)

In [ ]:
%matplotlib inline
plt.plot(rad_data[750,750,:])
plt.show()

In [ ]:
%matplotlib inline
plt.plot(rho[750,750,:])
plt.show()